In [1]:
!pip install -q ultralytics

In [2]:
# Импорт необходимых библиотек
from ultralytics import YOLO
import torch
import os
import pandas as pd
from tqdm.notebook import tqdm
import gc
from collections import defaultdict

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# Установка путей
DATASET_PATH = '.....'  # путь к данным
test_path = os.path.join(DATASET_PATH, 'test/images')  # путь к тестовым изображениям
MODEL_PATH = '.....'  # путь к сохраненной модели

In [4]:
test_files = sorted([f for f in os.listdir(test_path) if f.endswith(('.jpg', '.jpeg', '.png'))])
if not test_files:
    raise ValueError(f"В директории {test_path} нет изображений!")

print(f"Найдено {len(test_files)} изображений для обработки")

Найдено 2999 изображений для обработки


In [5]:
%%writefile /kaggle/working/waste.yaml
names:
- PET (transparent) (green)
- PET (transparent) (brown)
- PET (transparent) (blue)
- PET (transparent)
- PET (transparent) (dark blue)
- PET (black)
- PET (white)
- PET (sticker)
- PET (flacon)
- PET (household chemicals)
- PND (household chemicals)
- PND packet
- Other plastic
- Other plastic (transparent)
- Not plastic
nc: 15
path: /kaggle/input/renue-data/data_zip/renue-data
test: test/images
train: train/images
val: val/images

Writing /kaggle/working/waste.yaml


In [6]:
step = 8  # размер батча
device = '0' if torch.cuda.is_available() else 'cpu'


In [7]:
# Загрузка обученной модели
print("=== Загрузка модели ===")
model = YOLO(MODEL_PATH)
print("Модель загружена")

=== Загрузка модели ===
Модель загружена


In [8]:
# Инициализация словаря для хранения предсказаний по каждому изображению
predictions = defaultdict(list)

In [9]:
# Получение предсказаний
images_with_objects = 0
images_without_objects = 0
for i in tqdm(range(0, len(test_files), step), desc="Обработка батчей"):
    batch_files = test_files[i:i+step]
    batch_paths = [os.path.join(test_path, f) for f in batch_files]
    
    # Получаем предсказания для батча
    results = model.predict(
        source=batch_paths,
        imgsz=832,
        verbose=False,
        
        conf=0.25,
        device=device
    )
    
    # Обработка результатов
    for result in results:
        img_name = os.path.basename(result.path)
        if not img_name.endswith('.jpg'):
            img_name = img_name + '.jpg'
        
        boxes = result.boxes
        
        if len(boxes) > 0:
            images_with_objects += 1
            for box in boxes:
                cls_id = int(box.cls[0])
                conf = float(box.conf[0])
                bbox = box.xyxy[0].tolist()
                
                # Преобразуем координаты
                width = round(bbox[2] - bbox[0], 2)
                height = round(bbox[3] - bbox[1], 2)
                
                pred = [
                    float(cls_id + 1),
                    round(conf, 2),
                    round(bbox[0], 2),
                    round(bbox[1], 2),
                    width,
                    height
                ]
                predictions[img_name].append(pred)
        else:
            images_without_objects += 1
            predictions[img_name] = []

print("Обработка изображений завершена")

Обработка батчей:   0%|          | 0/375 [00:00<?, ?it/s]

Обработка изображений завершена


In [10]:
# Создание DataFrame
submission_df = pd.DataFrame({
    'file_name': list(predictions.keys()),
    'bbox': list(predictions.values())
})


In [11]:
# Сортировка по имени файла
submission_df = submission_df.sort_values('file_name').reset_index(drop=True)

In [12]:
# Сохранение результатов
output_path = '....' # укажите путь для сохранения результатов
submission_df.to_csv(output_path, index=False)
print(f"Результаты сохранены в {output_path}")

Результаты сохранены в /kaggle/working/submission.csv


In [13]:
# Вывод статистики
print(f"\nСтатистика обработки:")
print(f"Всего изображений: {len(test_files)}")
print(f"Изображений с объектами: {images_with_objects}")
print(f"Изображений без объектов: {images_without_objects}")


Статистика обработки:
Всего изображений: 2999
Изображений с объектами: 2935
Изображений без объектов: 64


In [14]:
# Вывод примера предсказаний
print("\nПример первых предсказаний:")
for i, row in submission_df.head().iterrows():
    print(f"\nФайл: {row['file_name']}")
    print(f"Найденные объекты: {row['bbox']}")


Пример первых предсказаний:

Файл: 000003.jpg
Найденные объекты: [[2.0, 0.86, 607.78, 433.98, 386.08, 382.06], [4.0, 0.69, 986.87, 671.35, 108.81, 173.13], [4.0, 0.68, 821.04, 0.0, 200.47, 228.34], [13.0, 0.54, 760.59, 363.11, 228.99, 225.48]]

Файл: 000006.jpg
Найденные объекты: [[4.0, 0.88, 799.99, 0.0, 215.18, 399.46], [2.0, 0.87, 611.16, 607.11, 384.82, 383.52], [4.0, 0.72, 989.1, 852.14, 106.65, 171.15], [13.0, 0.28, 765.05, 535.48, 225.87, 239.14]]

Файл: 000013.jpg
Найденные объекты: [[2.0, 0.86, 613.72, 1025.24, 378.06, 396.83], [4.0, 0.78, 727.61, 270.06, 278.54, 547.13], [1.0, 0.68, 693.32, 485.26, 154.21, 183.01], [4.0, 0.54, 987.62, 1263.81, 112.1, 179.87]]

Файл: 000016.jpg
Найденные объекты: [[1.0, 0.86, 694.56, 746.67, 181.11, 244.94], [2.0, 0.84, 616.02, 1209.09, 339.88, 276.57], [5.0, 0.42, 837.72, 467.55, 226.46, 543.48], [4.0, 0.39, 835.95, 469.45, 228.27, 542.79], [3.0, 0.32, 830.9, 464.35, 231.18, 544.43], [13.0, 0.31, 771.14, 1140.05, 224.73, 240.85]]

Файл: 0000